In [ ]:
import os
from tensorflow.python.client import device_lib
import tensorflow as tf

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu7 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu4 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu6 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu4# + "," + gpu6
print(get_available_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import cv2
import pickle
import time
import pandas as pd

In [ ]:
import ipywidgets as widgets
import matplotlib.animation as animation
from PIL import Image
from matplotlib import cm
import tqdm
import matplotlib.gridspec as gridspec

In [ ]:
from tensorflow.keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import History

from utils_metrics import *
from utils_visualize import *
from utils_loss import *
from model import *
from model_dropout_all import *

In [ ]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_DEPTH = 155

IMG_HEIGHT_UNET = 256
IMG_WIDTH_UNET = 256

N_IMG = 369
length_file = IMG_DEPTH * N_IMG
smooth = 1e-4

train_size = 250 * IMG_DEPTH
valid_size = 69 * IMG_DEPTH
test_size = 50 * IMG_DEPTH

train_begin = 0
train_stop = (250 * IMG_DEPTH) 
valid_begin = 250 * IMG_DEPTH
valid_stop = ((250 + 69) * IMG_DEPTH) 
test_begin = ((250 + 69) * IMG_DEPTH)
n_images = N_IMG * IMG_DEPTH 

n_channels = 3

CLASSES = ['0','1','2','3','4']
EPOCHS = 20
n_classes = 5

LR = 1e-4
BATCH_SIZE = 16
dropout = 0.2
reducing_factor = 8

In [ ]:
train_images = np.zeros((IMG_HEIGHT, IMG_WIDTH, train_size, n_channels))
valid_images = np.zeros((IMG_HEIGHT, IMG_WIDTH, valid_size, n_channels))

train_labels = np.zeros((IMG_HEIGHT, IMG_WIDTH, train_size))
valid_labels = np.zeros((IMG_HEIGHT, IMG_WIDTH, valid_size))

In [ ]:
test_images = np.zeros((IMG_HEIGHT, IMG_WIDTH, test_size, n_channels))
test_labels = np.zeros((IMG_HEIGHT, IMG_WIDTH, test_size))

In [ ]:
dataset_directory_t1 = os.path.join("..","data", 't1_images.hdf5')
dataset_directory_t1ce = os.path.join("..","data", 't1ce_images.hdf5')
dataset_directory_t2 = os.path.join("..","data", 't2_images.hdf5')
dataset_directory_flair = os.path.join("..","data", 'flair_images.hdf5')
dataset_directory_seg = os.path.join("..","data", 'seg_images.hdf5')

___

Load data and model

In [ ]:
train_images[...,1] = load_images_from_hdf5(dataset_directory_t1ce, train_begin, train_stop)
valid_images[...,1] = load_images_from_hdf5(dataset_directory_t1ce, valid_begin, valid_stop)

In [ ]:
train_images[...,2] = load_images_from_hdf5(dataset_directory_t2, train_begin, train_stop)
valid_images[...,2] = load_images_from_hdf5(dataset_directory_t2, valid_begin, valid_stop)

In [ ]:
train_images[...,0] = load_images_from_hdf5(dataset_directory_flair, train_begin, train_stop)
valid_images[...,0] = load_images_from_hdf5(dataset_directory_flair, valid_begin, valid_stop)

In [ ]:
train_labels = load_images_from_hdf5(dataset_directory_seg, train_begin, train_stop)
valid_labels = load_images_from_hdf5(dataset_directory_seg, valid_begin, valid_stop)

In [ ]:
"""
Test dataset
"""

test_images[...,0] = load_images_from_hdf5(dataset_directory_flair, test_begin, n_images)
test_images[...,1] = load_images_from_hdf5(dataset_directory_t1ce, test_begin, n_images)
test_images[...,2] = load_images_from_hdf5(dataset_directory_t2, test_begin, n_images)
test_labels = load_images_from_hdf5(dataset_directory_seg, test_begin, n_images)

In [ ]:
np.shape(valid_images)

In [ ]:
#control images

slice_num = 99

visualize(
    flair=train_images[...,slice_num,0].squeeze(),
    t1ce=train_images[...,slice_num,1].squeeze(),
    t2=train_images[...,slice_num,2].squeeze(),
    #flair=train_images[...,slice_num,3].squeeze(),
    seg=train_labels[...,slice_num].squeeze(),
)

In [ ]:
#control images

slice_num = 200

visualize(
    flair=valid_images[...,slice_num,0].squeeze(),
    t1ce=valid_images[...,slice_num,1].squeeze(),
    t2=valid_images[...,slice_num,2].squeeze(),
    #flair=valid_images[...,slice_num,3].squeeze(),
    seg=valid_labels[...,slice_num].squeeze(),
)

____

In [ ]:
class Dataset:
    
    def __init__(self, images_train, images_label, classes = CLASSES):
        
        self.images_fps = images_train
        self.masks_fps = images_label
        self.classes = classes
        self.class_values = [self.classes.index(cls.lower()) for cls in self.classes]
        self.ids = np.shape(self.images_fps)[2]
    
    def __getitem__(self, i):
        
        #initialize matrices to desired shape, unet shape
        X_new = np.ones((IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_channels)) * self.images_fps[3,3,i,0] #background value
        y_new = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET))
        
        #fit images into initialized matrices
        X_new[:IMG_HEIGHT,:IMG_HEIGHT,0] = self.images_fps[:,:,i,0]
        X_new[:IMG_HEIGHT,:IMG_HEIGHT,1] = self.images_fps[:,:,i,1]
        X_new[:IMG_HEIGHT,:IMG_HEIGHT,2] = self.images_fps[:,:,i,2]
        
        #commented in order to keep 3 instead of 4 channels
        #X_new[:IMG_HEIGHT,:IMG_HEIGHT,3] = self.images_fps[:,:,i,3]
               
        y_new[:IMG_HEIGHT,:IMG_HEIGHT] = self.masks_fps[:,:,i]
        
        #return function and one-hot-encoded mask
        return X_new, tf.keras.utils.to_categorical(y_new, num_classes=n_classes)
    
    def __len__(self):
        
        return self.ids

In [ ]:
class Dataloder(tf.keras.utils.Sequence):
    
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=BATCH_SIZE, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __getitem__(self, index):
        
        #just to make sure
        if "data" in dir():
            del data
        if "batch" in dir():
            del batch
        
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [np.arange(len(self.dataset))[k] for k in indexes]

        data = []
        batch = []
        
        for j in list_IDs_temp:

            data.append(self.dataset[j])
        
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        

        return (batch[0],batch[1])
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.indexes) / self.batch_size))
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        self.indexes = np.arange(len(self.dataset))
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)  

In [ ]:
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    dice_inner_0, 
    dice_inner_1, 
    dice_inner_2, 
    dice_inner_3, 
    dice_inner_4,
]

In [ ]:
weights = [0.1,2.3,1.8,0.01,1.8]
#weights = None

In [ ]:
def average(x, per_image=False, class_weights=None, **kwargs):
    if per_image:
        x = K.mean(x, axis=0)
    if class_weights is not None:
        x = x * class_weights
    return K.mean(x)

def _gather_channels(x, indexes, **kwargs):
    """Slice tensor along channels axis by given indexes"""
    backend = K
    if backend.image_data_format() == 'channels_last':
        x = backend.permute_dimensions(x, (3, 0, 1, 2))
        x = backend.gather(x, indexes)
        x = backend.permute_dimensions(x, (1, 2, 3, 0))
    else:
        x = backend.permute_dimensions(x, (1, 0, 2, 3))
        x = backend.gather(x, indexes)
        x = backend.permute_dimensions(x, (1, 0, 2, 3))
    return x

def get_reduce_axes(per_image, **kwargs):
    backend = K
    axes = [1, 2] if backend.image_data_format() == 'channels_last' else [2, 3]
    if not per_image:
        axes.insert(0, 0)
    return axes

def gather_channels(*xs, indexes=None, **kwargs):
    """Slice tensors along channels axis by given indexes"""
    if indexes is None:
        return xs
    elif isinstance(indexes, (int)):
        indexes = [indexes]
    xs = [_gather_channels(x, indexes=indexes, **kwargs) for x in xs]
    return xs

def round_if_needed(x, threshold, **kwargs):
    backend = K
    if threshold is not None:
        x = backend.greater(x, threshold)
        x = backend.cast(x, backend.floatx())
    return x

In [ ]:
def f_score_loss(gt, pr, beta=1, class_weights=weights, class_indexes=None, smooth=smooth, per_image=False, threshold=None, **kwargs):
    gt, pr = gather_channels(gt, pr, indexes=class_indexes, **kwargs)
    pr = round_if_needed(pr, threshold, **kwargs)
    axes = get_reduce_axes(per_image, **kwargs)

    # calculate score
    tp = K.sum(gt * pr, axis=axes)
    fp = K.sum(pr, axis=axes) - tp
    fn = K.sum(gt, axis=axes) - tp

    score = ((1 + beta ** 2) * tp + smooth) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)
    score = average(score, per_image, class_weights, **kwargs)

    return 1-score

In [ ]:
def iou_score_loss(gt, pr, class_weights=weights, class_indexes=None, smooth=smooth, per_image=False, threshold=None, **kwargs):
    backend = K
    gt, pr = gather_channels(gt, pr, indexes=class_indexes, **kwargs)
    pr = round_if_needed(pr, threshold, **kwargs)
    axes = get_reduce_axes(per_image, **kwargs)

    # score calculation
    intersection = backend.sum(gt * pr, axis=axes)
    union = backend.sum(gt + pr, axis=axes) - intersection

    score = (intersection + smooth) / (union + smooth)
    print(score)
    score = average(score, per_image, class_weights, **kwargs)

    return 1-score


In [ ]:

#loss = weighted_categorical_crossentropy(weights)
# loss = dice_niftynet
loss = f_score_loss
optim = tf.keras.optimizers.Adam(LR)

In [ ]:
model = unet(
    pretrained_weights = None,
    input_size = (IMG_HEIGHT_UNET,IMG_WIDTH_UNET,n_channels),
    dropout = dropout,
    reducing_factor = reducing_factor,
    n_classes = n_classes)
model.compile(optim, loss=loss, metrics = METRICS)
model.summary()

In [ ]:
model_dropout_all = unet_dropout_all(
    pretrained_weights = None,
    input_size = (IMG_HEIGHT_UNET,IMG_WIDTH_UNET,n_channels),
    dropout = dropout,
    reducing_factor = reducing_factor,
    n_classes = n_classes)
model_dropout_all.compile(optim, loss=loss, metrics = METRICS)
model_dropout_all.summary()

In [ ]:
# tf.debugging.set_log_device_placement(True)
# with tf.device('/CPU:0'):

train_dataset = Dataset(train_images, train_labels, classes=CLASSES)
valid_dataset = Dataset(valid_images, valid_labels, classes=CLASSES)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_channels)
assert train_dataloader[0][1].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_classes)

# define callbacks for learning rate scheduling and best checkpoints saving
# callbacks = [
#     tf.keras.callbacks.ModelCheckpoint('./best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
#     tf.keras.callbacks.ReduceLROnPlateau(),
# ]

# callbacks = [
#     tf.keras.callbacks.ReduceLROnPlateau(),
# ]

In [ ]:
class printbatch(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, epoch, logs={}):

        res_dir = os.path.join("..","data","test_images")

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))
        
# pb = printbatch()        

In [ ]:
# checkpoint_path =  os.path.join("..","data","checkpoints","training_0211/cp-{epoch:04d}.ckpt")
# checkpoint_dir = os.path.dirname(checkpoint_path)

# STEPS_PER_EPOCH = train_stop / BATCH_SIZE
# SAVE_PERIOD = 5

# # Create a callback that saves the model's weights every 5 epochs
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     save_weights_only=True,
#     save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH)
#     )

# # history_callback = tf.keras.callbacks.CSVLogger('history.csv',separator=',',append=False)

# # callbacks = [pb, cp_callback]


In [ ]:
image, mask = valid_dataset[100]

visualize(
    image=image[...,0], ### if putting the whole image, black and white, could it be the reason of the problem??
    no_tumor=mask[..., 0].squeeze(),
    one=mask[..., 1].squeeze(),
    two=mask[..., 2].squeeze(),
    three=mask[..., 3].squeeze(),
    four=mask[..., 4].squeeze(),
)

In [ ]:
SOF = 99
image, mask = train_dataset[SOF]

visualize(
    image_flair=image[...,0], ### if putting the whole image, black and white, could it be the reason of the problem??
    image_t1ce=image[..., 1].squeeze(),
    image_t2=image[..., 2].squeeze(),
    data_flair=train_images[..., SOF,0].squeeze(),
    data_t1ce=train_images[..., SOF, 1].squeeze(),
    data_t2=train_images[..., SOF,2].squeeze(),
)

In [ ]:
history = model.fit(
    train_dataloader,
    epochs=EPOCHS,
    verbose = 1,
    validation_data=valid_dataloader, 
)

In [ ]:
history_dropout_all = model_dropout_all.fit(
    train_dataloader,
    epochs=EPOCHS,
    verbose = 1,
    validation_data=valid_dataloader, 
)

### Save checkpoint
___

In [ ]:
checkpoint_path= os.path.join("..","data","checkpoints","simple_unet_20_epochs_50_dropout_all.ckpt")
history = history_dropout_all

### Visualizing results

In [ ]:
import seaborn as sn
sn.set(font_scale=1.0)

visualize_histories(
    class_zero = (history.history['dice_inner_0'], history.history['val_dice_inner_0']), ### if putting the whole image, black and white, could it be the reason of the problem??
    class_one = (history.history['dice_inner_1'], history.history['val_dice_inner_1']),
    class_two = (history.history['dice_inner_2'], history.history['val_dice_inner_2']),
    class_three = (history.history['dice_inner_3'], history.history['val_dice_inner_3']),
    class_four = (history.history['dice_inner_4'], history.history['val_dice_inner_4'])
)

In [ ]:
print("class 0: ", history.history['dice_inner_0'][9])
print("class 0 val: ", history.history['val_dice_inner_0'][9])
print("class 1: ", history.history['dice_inner_1'][9])
print("class 1 val: ", history.history['val_dice_inner_1'][9])
print("class 2: ", history.history['dice_inner_2'][9])
print("class 2 val: ", history.history['val_dice_inner_2'][9])
print("class 3: ", history.history['dice_inner_3'][9])
print("class 3 val: ", history.history['val_dice_inner_3'][9])
print("class 4: ", history.history['dice_inner_4'][9])
print("class 4 val: ", history.history['val_dice_inner_4'][9])

In [ ]:
def FPR(FP,TN):
    return FP/(FP + TN)

def FNR(FN,TP):
    return FN/(FN + TP)


sensitivity_train = Sensitivity(np.array(history.history['tp']),np.array(history.history['fn']))
specificity_train = Specificity(np.array(history.history['tn']),np.array(history.history['fp']))
FPR_train = FPR(np.array(history.history['fp']),np.array(history.history['tn']))
FNR_train = FNR(np.array(history.history['fn']),np.array(history.history['tp']))
dice_score_train = DiceScore([history.history['dice_inner_0'],history.history['dice_inner_1'],history.history['dice_inner_2'],history.history['dice_inner_4']])

print("Train: sensitivity: ", sensitivity_train[-1], ", specificity: ", specificity_train[-1], ", dice score: ", dice_score_train)

sensitivity_val = Sensitivity(np.array(history.history['val_tp']),np.array(history.history['val_fn']))
specificity_val = Specificity(np.array(history.history['val_tn']),np.array(history.history['val_fp']))
FPR_train = FPR(np.array(history.history['val_fp']),np.array(history.history['val_tn']))
FNR_train = FNR(np.array(history.history['val_fn']),np.array(history.history['val_tp']))
dice_score_val = DiceScore([history.history['val_dice_inner_0'],history.history['val_dice_inner_1'],history.history['val_dice_inner_2'],history.history['val_dice_inner_4']])

print("Validation: sensitivity: ", sensitivity_val[-1], ", specificity: ", specificity_val[-1], ", dice score: ", dice_score_val)

total_pixels_train = np.array(history.history['tp'])[-1] + np.array(history.history['fp'])[-1] + np.array(history.history['fn'])[-1] + np.array(history.history['tn'])[-1]
total_pixels_val = np.array(history.history['val_tp'])[-1] + np.array(history.history['val_fp'])[-1] + np.array(history.history['val_fn'])[-1] + np.array(history.history['val_tn'])[-1]

confusion_mat_train = [[np.array(history.history['tp'])[-1],np.array(history.history['fp'])[-1]],[np.array(history.history['fn'])[-1],np.array(history.history['tn'])[-1]]]
confusion_mat_val = [[np.array(history.history['val_tp'])[-1],np.array(history.history['val_fp'])[-1]],[np.array(history.history['val_fn'])[-1],np.array(history.history['val_tn'])[-1]]]/total_pixels_val



In [ ]:
#Plotting Confussion matrix

df_cm = pd.DataFrame(confusion_mat_val, range(2), range(2))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt=".2%", xticklabels=["1","0"], yticklabels=["1","0"]) # font size
plt.xlabel('true')
plt.ylabel('prediction')
plt.title('Confusion matrix Validation set')

plt.show()

In [ ]:
#Plotting loss

sn.set(font_scale=1.0)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.ylim([0,1])
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
n = 1
# ids = np.random.choice(np.arange(len(labels_train)), size=n)
ids = [100]
for i in ids:
    
    image, gt_mask = valid_dataset[i]
    #image = np.expand_dims(image, axis=0)
    pr_mask = model_dropout_all.predict(image[np.newaxis, ...])
    visualize_im('RdYlGn',
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,0].squeeze(),
        pr_mask=pr_mask[...,0].squeeze(),
        difference = gt_mask[...,0].squeeze() - pr_mask[...,0].squeeze(),
    )
    visualize_im('RdYlGn',
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,1].squeeze(),
        pr_mask=pr_mask[...,1].squeeze(),
        difference = gt_mask[...,1].squeeze() - pr_mask[...,1].squeeze(),
    )
    visualize_im('RdYlGn',
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,2].squeeze(),
        pr_mask=pr_mask[...,2].squeeze(),
        difference = gt_mask[...,2].squeeze() - pr_mask[...,2].squeeze(),
    )
    visualize_im('RdYlGn',
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,4].squeeze(),
        pr_mask=pr_mask[...,4].squeeze(),
        difference = gt_mask[...,4].squeeze() - pr_mask[...,4].squeeze(),
    )

### Saving weights
___

In [ ]:
model_dropout_all.save_weights(checkpoint_path)

### Testing
___

In [ ]:
test_dataset = Dataset(test_images, test_labels, classes=CLASSES)
test_dataloader = Dataloder(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
assert test_dataloader[0][0].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_channels)
assert test_dataloader[0][1].shape == (BATCH_SIZE, IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_classes)

In [ ]:
# Restore the weights
# checkpoint_path = os.path.join('..', 'data', 'checkpoints')
# checkpoint_number = 'cp-0040.ckpt'
# checkpoint_path = os.path.join(checkpoint_path, checkpoint_number)
model.load_weights(checkpoint_path)

In [ ]:
results = model_dropout_all.evaluate(test_dataloader, verbose=1)

In [ ]:
test_dice_score = DiceScore([results[5],results[6],results[7],results[9]])
print(test_dice_score)
print(model.metrics)

In [ ]:
results[5]

In [ ]:
#https://github.com/pablosarricolea/Deep-learning-with-Python/blob/master/Notebooks/Keract-activation.ipynb
from keract import get_activations, display_activations, display_heatmaps

image, gt_mask = test_dataset[100]
plt.imshow(image[...,1])
plt.grid(False)

In [ ]:
#Activation network
x = image[np.newaxis,...]
activations = get_activations(model, x)
display_activations(activations, save=False)

In [ ]:
#Heatmap, currenlty not working
display_heatmaps(activations, image, save=False)

___

In [ ]:
def single_dice_coef(y_true, y_pred_bin):
    # shape of y_true and y_pred_bin: (height, width)
    intersection = np.sum(y_true * y_pred_bin)
    if (np.sum(y_true)==0) and (np.sum(y_pred_bin)==0):
        return 1
    return (2*intersection) / (np.sum(y_true) + np.sum(y_pred_bin))

def dice_single_single(true,pred):
    true = np.ndarray.flatten(true)
    pred = np.ndarray.flatten(pred)
    pred = np.round(pred)

    intersection = np.sum(true * pred, axis=-1)
    true = np.sum(true, axis=-1)
    pred = np.sum(pred, axis=-1)

    return ((2*intersection) + 0.00001) / (true + pred + 0.000001)

def mean_dice_coef(y_true, y_pred_bin):
    # shape of y_true and y_pred_bin: (n_samples, height, width, n_channels)
    batch_size = y_true.shape[0]
    channel_num = y_true.shape[-1]
    mean_dice_channel = 0.
    for i in range(batch_size):
        for j in range(channel_num):
            channel_dice = single_dice_coef(y_true[i, :, :, j], y_pred_bin[i, :, :, j])
            mean_dice_channel += channel_dice/(channel_num*batch_size)
        print(mean_dice_channel)
    return mean_dice_channel

In [ ]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img

def plot_img(array, i):
    plt.imshow(array[i])
    plt.grid(False)
    
def plot_img_with_all_labels(array, image, i):
    channel_num = array.shape[-1]
    plt.imshow(array[i,...,1],cmap = 'Greens', alpha = 0.8)
    plt.imshow(array[i,...,2],cmap = 'Reds', alpha = 0.5)
    plt.imshow(array[i,...,4],cmap = 'Blues', alpha = 0.2)
    plt.imshow(image[i,...], alpha = 0.3, cmap = 'Greys')
    plt.grid(False)
    
    #to be able to make the gif later
    fig = plt.gcf()
    plt.xticks([])
    plt.yticks([])
    plt.close()
    return fig
    
def plot_contour(array,i):
    plt.contour(np.flipud(array[i]), [0.1, 0.5, 0.9], cmap = 'RdYlGn')

def plot_contour_with_gt(array, gt, image, i):
    plt.imshow(image[i], alpha = 0.3, cmap = 'Greys')
    plt.contour(gt[i],alpha = 0.5,cmap = 'gray')
    #plt.contour(array[i], [0.1, 0.9], cmap = 'RdYlGn')
    
    plt.contour(array[i,...,1],cmap = 'Greens', alpha = 0.8)
    plt.contour(array[i,...,2],cmap = 'Reds', alpha = 0.5)
    plt.contour(array[i,...,4],cmap = 'Blues', alpha = 0.2)
    
    #to be able to make the gif later
    fig = plt.gcf()
    plt.xticks([])
    plt.yticks([])
    plt.close()
    return fig
    

In [ ]:
image, _ =test_dataset[100]
y_probas = np.stack([model_dropout_all(image[np.newaxis,...], training=True) for sample in range(100)])
y_probas = np.squeeze(y_probas)
vis_img = np.ones((100,256,256))
for i in range(100):
    vis_img[i,...] = image[...,0]

In [ ]:
widgets.interact(plot_img_with_all_labels,
                array = widgets.fixed(y_probas),
                image = widgets.fixed(vis_img),
                i = widgets.IntSlider(value=0,min=0,max =(100-1),step=1,description='Frame N:',orientation='horizontal'))

In [ ]:
img = []

for i in tqdm.tqdm(range(100)):
    img.append(fig2img(plot_img_with_all_labels(y_probas, vis_img, i)))
    

In [ ]:
img[0].save('individual_predictions.gif',
               save_all=True, append_images=img[1:], optimize=False, duration=100, loop=0)

In [ ]:
volume_number = 2
simulation_number = 10
count = 0

#initialize saving volumes
simulation_array = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_classes, IMG_DEPTH, simulation_number))
gt_volume = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_classes, IMG_DEPTH))
image_volume = np.zeros((IMG_HEIGHT_UNET, IMG_WIDTH_UNET, n_channels, IMG_DEPTH))
dummy_dice = np.zeros((IMG_DEPTH,simulation_number))

#iterate across a whole volume
for j in range(IMG_DEPTH * (volume_number - 1), IMG_DEPTH * volume_number ):
    
    image, gt_mask = test_dataset[j]
    gt_volume[...,count] = gt_mask
    image_volume[...,count] = image
    
    #running predictions and saving results
    for i in range(simulation_number):
        
        simulation_array[...,count,i] = model_dropout_all(image[np.newaxis,...], training=True)
        
    count += 1

#obtaining the mean of the predictions
probability_map_mean = np.mean(simulation_array,axis=4)

In [ ]:
#volume dice coefficient

a = np.zeros((4))
for i in range(4):
    x = np.delete(gt_volume[np.newaxis,...,],3,axis = 3)[0,:,:,i,...]
    y = np.delete(probability_map_mean[np.newaxis,...,],3,axis = 3)[0,:,:,i,...]
    a[i] = single_dice_coef(y,x)
    
final_dice = np.mean(a)

In [ ]:
final_dice

In [ ]:
#setting arrays for their widget visualization

list_frames_visualization = np.rollaxis(probability_map_mean,3,0)
gt_volume_visualization = np.rollaxis(gt_volume,3,0)
image_volume_visualization = np.rollaxis(image_volume,3,0)

In [ ]:
channel_of_interest = 2
label_of_interest = 4

corrected_image = np.around(list_frames_visualization,2)
thing = gt_volume_visualization[...,label_of_interest]
for i in range(155):
    corrected_image[i,1:3,1:3] = 0.5 
    thing[1,1:3,1:3] = 0.1

widgets.interact(plot_contour_with_gt,
                array = widgets.fixed(corrected_image),
                gt = widgets.fixed(thing),
                image = widgets.fixed(image_volume_visualization[...,channel_of_interest]),
                i = widgets.IntSlider(value=0,min=0,max =(len(list_frames_visualization)-1),step=1,description='Frame N:',orientation='horizontal'))

In [ ]:
img = []

for i in tqdm.tqdm(range(len(list_frames_visualization[...,2]))):
    img.append(fig2img(plot_contour_with_gt(corrected_image, thing, image_volume_visualization[...,0],i)))
    

In [ ]:
img[0].save('volume_predictions.gif',
               save_all=True, append_images=img[1:], optimize=False, duration=100, loop=0)

___ 

### Network activation

In [ ]:
import visualkeras
from PIL import ImageFont

In [ ]:
#font = ImageFont.truetype('arial.pil')
font = ImageFont.load_default()
visualkeras.layered_view(model_dropout_all, legend = True, scale_xy=2, scale_z=0.5, font=font)

Image visualization of training